<font color='IndianRed'>
<font size=9> 

**Bases de Datos con PostgreSQL**

<font color = 'DodgerBlue'>
<font size = 5>
    
**Producto cartesiano.**

<font color = 'Black'>
<font size = 3>

Consideremos dos conjuntos finitos (en realidad esta restricción solo es práctica) $A$ y $B$. Se define su **producto cartesiano** como el conjunto $A\times B=\{(a,b)\,:\,a\in A\mbox{ y }b\in B\}$.

Es decir, el **producto cartesiano** son todas las parejas que se pueden formar donde la primera entrada es cualquier elemento de $A$ y la segunda entrada es cualquier elemento de $B$. Luego, si $A$ tiene $n$ elementos y $B$ tiene $m$ elementos, entonces el producto cartesiano de ambos conjuntos tiene $n \cdot m$ elementos.

Si $A = \{'Hola', 'Crayola'\}$ y $B = \{1, 4, 2, 6\}$. Entonces $A \times B = \{('Hola', 1), ('Hola', 4), ('Hola', 2), ('Hola', 6), ('Crayola', 1), ('Crayola', 4), ('Crayola', 2), ('Crayola', 6)\}$

---

<font color = 'Black'>
<font size = 3>

Consideremos las siguientes tablas:

**Tabla camisas**

|Id_camisa|camisa|Precio|
|--|--|--|
|1|lino blanco|210|
|2|algodon naranja|290|
|3|seda negra|260|

**Tabla pantalones**

|Id_pantalon|pantalon|Precio|
|--|--|--|
|1|mezclilla azul|470|
|2|mezclilla negra|730|


Si nos preguntáramos de cuántas maneras nos podemos vestir utilizando las prendas anteriores, es claro que la respuesta es 6. Todas las posibles maneras de hacerlo nos la da la tabla formada por el producto cartesiano de ambas tablas, considerando que las tablas son conjuntos de registros.

Esto nos da como salida la tabla

**Tabla camisas x pantalones**

|Id_camisa|camisa|Precio|Id_pantalon|pantalon|Precio|
|--|--|--|--|--|--|
|1|lino blanco|210|1|mezclilla azul|470|
|1|lino blanco|210|2|mezclilla negra|730|
|2|algodon naranja|210|1|mezclilla azul|470|
|2|algodon naranja|210|2|mezclilla negra|730|
|3|seda negra|210|1|mezclilla azul|470|
|3|seda negra|210|2|mezclilla negra|730|


Más aún, supongamos que no nos interesaban los Id, de modo que podemos hacer el producto cartesiano 
$$\Pi_{\mbox{Col2,Col3}}(\mbox{camisas})\times\Pi_{\mbox{Col2,Col3}}(\mbox{pantalones})$$


**Tabla $\Pi_{\mbox{Col2,Col3}}(\mbox{camisas})\times\Pi_{\mbox{Col2,Col3}}(\mbox{pantalones})$**

|camisa|Precio|pantalon|Precio|
|--|--|--|--|
|lino blanco|210|mezclilla azul|410|
|lino blanco|210|mezclilla negra|370|
|algodon naranja|210|mezclilla azul|410|
|algodon naranja|210|mezclilla negra|370|
|seda negra|210|mezclilla azul|410|
|seda negra|210|mezclilla negra|370|

<font color = 'Black'>
<font size = 3>

Supongamos ahora que queremos saber el precio total de cada combinación. Se nos presenta el siguiente problema: en la tabla anterior tenemos dos campos con el mismo nombre (Precio).

A este tipo de problemas se les conoce como ambigüedades. Para evitarlos, siempre que se hace un producto cartesiano de tablas se sugiere nombrar cada columna en formato *nombre_de_la_tabla.nombre_de_la_columna*. 

En particular,

```sql
create table camisas_pantalones as
select camisas.camisa, 
       camisas.precio as Precio_camisa, 
       pantalones.pantalon, 
       pantalones.precio as Precio_pantalon, 
       camisas.precio + pantalones.precio as Precio_final
from camisas, pantalones;
```

significa, literalmente, crear una tabla llamada *camisas_pantalones* cuyas columnas serán:

* la columna camisa de la tabla camisas (camisas.camisa).
* la columna precio de la tabla camisas (camisas.precio), la cual se llamará *Precio_camisa*.
* la columna pantalon de la tabla pantalones (pantalones.pantalon).
* la columna precio de la tabla pantalones (pantalones.precio), la cual se llamará *Precio_pantalon*.
* Una nueva columna que contendrá la suma respectiva de los precios de camisas con pantalones, la cual se llamará *Precio_final*.

En la siguiente imagen se puede ver la manera en cómo se implementa en pgAdmin la solución del problema anterior:

<img src = "Img/sesion_11_6.png">

---

<font color = 'DodgerBlue'>
<font size = 5>
    
**EXTRA: La función concatenado: concat**

<font color = 'Black'>
<font size = 3>

En la tabla anterior ya tenemos todas las combinaciones disponibles junto con el precio de cada una de ellas. Nos gustaría sin embargo presentar la información de una manera más estilizada.

``CONCAT`` es una función que concatena datos de tipo cadena dando como resultado una única cadena:

```sql
select concat('Camisa de ', camisas_pantalones.camisa,
              ' con pantalón de ', camisas_pantalones.pantalon) 
                  as combinacion, 
       Precio_final
from camisas_pantalones 
order by Precio_final desc
```

La instrucción anterior nos da como resultado la siguiente tabla:

|Combinacion|Precio_final|
|--|--|
|Camisa de algodón naranja con pantalón de mezclilla negra|1020|
|Camisa de seda negra con pantalón de mezclilla negra|990|
|Camisa de lino blanco con pantalón de mezclilla negra|940|
|Camisa de algodón naranja con pantalón de mezclilla azul|760|
|Camisa de seda negra con pantalón de mezclilla azul|730|
|Camisa de lino blanco con pantalón de mezclilla azul|680|

---

<font color = 'DodgerBlue'>
<font size = 5>
    
**Joins**

<font color = 'Black'>
<font size = 3>

Join se usa para combinar columnas de una o más tablas en función de los valores de las columnas comunes entre tablas relacionadas. Las columnas comunes suelen ser las columnas de clave principal de la primera tabla y las columnas de clave externa de la segunda tabla. Sin embargo, esto no tiene por qué ser así. Llamamos columna join a las columnas que usaremos para hacer la operación join.

En esta sección estudiaremos cinco tipos de Join.

Consideremos las siguientes tablas:

**Tabla A**

|ID_A|Fruta|
|--|--|
|1|Manzana|
|2|Naranja|
|3|Plátano|
|4|Pepino|

**Tabla B**

|ID_B|Fruta|
|--|--|
|1|Naranja|
|2|Manzana|
|3|Sandia|
|4|Pera|

---

<font color = 'DodgerBlue'>
<font size = 5>
    
**Inner Join**

<font color = 'Black'>
<font size = 3>

El inner join es un join *de intersección*. Si $S$ y $T$ son tablas, su inner join, toma cada registro en $S$ y compara su columna join con la columna join de $T$. Cada vez que halle una coincidencia, se crea un registro formado por los dos registros respectivos donde halló la coincidencia.

Por ejemplo, el inner join de las tablas A y B anteriores, usando columna join de A a Fruta y columna join de B a Fruta de B, recorrerá cada fila de A buscando coincidir Fruta de A con Fruta de B para formar una nueva tabla cuyos renglones son la combinación de los renglones de A con los renglones de B donde hay coincidencia, dando como resultado la tabla

**A inner Join B por fruta de A y fruta de B**

|ID_A|Fruta|ID_B|Fruta|
|--|--|--|--|
|1|Manzana|2|Manzana|
|2|Naranja|1|Naranja|

El diagrama de Venn para esta situación es el siguiente:

<img src = "Img/inner_join.png" width="300">

---

<font color = 'DodgerBlue'>
<font size = 5>
    
**Left Outer Join**

<font color = 'Black'>
<font size = 3>

La unión izquierda (*left join*) comienza a seleccionar datos de la tabla izquierda. Compara los valores en la columna de Join de la tabla izquierda con los valores en la columna join de la tabla derecha. Si estos valores son iguales, left join crea una nueva fila que contiene columnas de ambas tablas y agrega esta nueva fila al conjunto de resultados. En caso de que los valores no sean iguales, left join también crea una nueva fila que contiene columnas de ambas tablas y la agrega al conjunto de resultados llenando las columnas correspondientes a la tabla derecha con nulo.

Por ejemplo, si A es la tabla izquierda y B es la derecha, entonces el left join de A con B usando como columnas join las frutas nos da como resultado

|Id_A|Fruta|Id_B|Fruta|
|--|--|--|--|
|1|Manzana|2|Manzana|
|2|Naranja|1|Naranja|
|3|Plátano|Null|Null|
|4|Pepino|Null|Null|

El diagrama de Venn correspondiente al left join como lo hemos descrito es

<img src = "Img/left_join_null.png" width="300">

En este caso, se trae toda la información.

<font color = 'Black'>
<font size = 3>

Sin embargo, si nos interesa quedarno exclusivamente con aquellos registros que no encontraron correspondencia en la tabla derecha, podemos añadir un filtro sobre los valores nulos, lo cual nos da como resultado

|Id_A|Fruta|Id_B|Fruta|
|--|--|--|--|
|3|Plátano|Null|Null|
|4|Pepino|Null|Null|

Este resultado se presenta de la siguiente manera:

<img src="Img/left_join_only.png" width="300">

Es decir, cuando yo quiero aquellos valores que no tuvieron coincidencia en la tabla derecha, que serían plátano y pepino.

Este caso es como el de una resta, elementos que están en un conjunto pero no están en el otro. En SQL se puede usar un 'WHERE'.

---

<font color = 'DodgerBlue'>
<font size = 5>
    
**Right Outer join**

<font color = 'Black'>
<font size = 3>

El Right Outer join es la versión "derecha" del left outer join. Funciona de la misma manera pero de derecha a izquierda.

Por ejemplo, si A es la tabla izquierda y B es la derecha, entonces el left join de A con B usando como columnas join las frutas nos da como resultado

|Id_A|Fruta|Id_B|Fruta|
|--|--|--|--|
|2|Naranja|1|Naranja|
|1|Manzana|2|Manzana|
|Null|Null|3|Sandía|
|Null|Null|4|Pera|

El diagrama de Venn correspondiente a este join es

<img src = "Img/right_join_null.png" width="300">

<font color = 'Black'>
<font size = 3>

Al igual que en el caso de left join, si únicamente nos interesan aquellos registros donde no se halló coincidencia, basta con hacer un filtro sobre los null para obtener la siguiente tabla

|Id_A|Fruta|Id_B|Fruta|
|--|--|--|--|
|Null|Null|3|Sandía|
|Null|Null|4|Pera|

Es decir

<img src = "Img/right_join_only.png" width="300">

---

<font color = 'DodgerBlue'>
<font size = 5>
    
**Full join**

<font color = 'Black'>
<font size = 3>

La combinación externa completa (full outer join) devuelve un conjunto de resultados que contiene todas las filas de las tablas izquierda y derecha, con las filas coincidentes de ambos lados si están disponibles. En caso de que no haya coincidencia, las columnas de la tabla se rellenarán con NULL.

Por ejemplo, si A es la tabla izquierda y B es la derecha, entonces el left join de A con B usando como columnas join las frutas nos da como resultado

|Id_A|Fruta|Id_B|Fruta|
|--|--|--|--|
|1|Manzana|2|Manzana|
|2|Naranja|1|Naranja|
|3|Plátano|Null|Null|
|4|Pepino|Null|Null|
|Null|Null|3|Sandía|
|Null|Null|4|Pera|

El diagrama de Venn correspondientes es 

<img src = "Img/full_join_null.png" width="300">

<font color = 'Black'>
<font size = 3>

Nuevamente, si solo nos interesan los registros que no encontraron coincidencias, hacemos un filtro sobre los nulos (tanto de A como de B). El resultado en el ejemplo es

|Id_A|Fruta|Id_B|Fruta|
|--|--|--|--|
|3|Plátano|Null|Null|
|4|Pepino|Null|Null|
|Null|Null|3|Sandía|
|Null|Null|4|Pera|

<img src = "Img/full_join_only.png" width="300">

---

<font color = 'Black'>
<font size = 3>

Concluimos esta sección con un breve resumen de los join, que hemos mostrado, que además muestra la sintaxis para Postgresql.

<img src = "Img/joins.png" width="700">